In [60]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [2]:
credit=pd.read_csv("C:\\Users\\kaust\\Desktop\\Credit card spend prediction case study\\CREDIT CARD USERS DATA.csv",encoding="latin-1")

C:\Users\kaust\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2,19,21,49,87,88,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
credit.drop(['custid',"birthmonth"],axis=1,inplace=True)

In [4]:
## custid and birthmonth are irrelevant variables, hence they are dropped.

In [5]:
## Numerical and Categorical variables :--

In [6]:
cat_var=['region','townsize','gender','agecat','edcat','jobcat','union','employ','empcat','retire','inccat',
         'default','jobsat','marital','spousedcat','homeown','hometype','address','addresscat','cars','carown','cartype',
         'carcatvalue','carbought','carbuy','commute','commutecat','commutecar','commutemotorcycle','commutecarpool',
         'commutebus','commuterail','commutepublic','commutebike','commutewalk','commutenonmotor','telecommute','reason',
         'polview','polparty','polcontrib','vote','card','cardtype','cardbenefit','cardfee','cardtenure','cardtenurecat',
         'card2','card2type','card2benefit','card2fee','card2tenure','card2tenurecat','active','bfast','churn','tollfree',
         'equip','callcard','wireless','multline','voice','pager','internet','callid','callwait','forward','confer','ebill',
         'owntv','ownvcr','owndvd','owncd','ownpda','ownpc','ownipod','owngame','ownfax','news','response_01','response_02',
         'response_03']


# Numerical List--
numerical_var=[]
for var in credit.columns:
    if var not in cat_var:
        numerical_var.append(var)


# Updating the given list anytime while droping variables.

def list_update(var):
    if var in cat_var:
        cat_var.remove(var)
    elif var in numerical_var:
        numerical_var.remove(var)
    else:
        print("Variable not present in either of the list.")

def list_drop(listx):
    for var in listx:
        list_update(var)
        credit.drop(var,axis=1,inplace=True)

In [7]:
credit[credit == '#NULL!'] = np.nan
credit=credit.astype(float)

C:\Users\kaust\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [8]:
credit['total_spent']=credit['cardspent']+credit['card2spent']
credit['total_spent_ln']=np.log(credit['total_spent'])

# Adding given variable to the numerical list
templist=['total_spent_ln']
for var in templist:
    numerical_var.append(var)

dplist=['total_spent','cardspent','card2spent']
list_drop(dplist)

Variable not present in either of the list.


In [9]:
## Missing Value Treatment

In [10]:
credit.isnull().sum().value_counts()

0       114
2         3
3296      2
3656      2
2622      2
3         2
1         2
1422      1
1419      1
dtype: int64

In [11]:
mis=credit.isnull().sum()
print("Variable with Percentage of Missing Value---")
totalx=5000.00
for var in mis.index:
    t=mis[var]/totalx
    t=t*100
    if t>= 25.00:
        list_update(var)
        print('{0}:{1}'.format(var,t))
        credit.drop(var,axis=1,inplace=True)

Variable with Percentage of Missing Value---
lntollmon:52.44
lntollten:52.44
lnequipmon:65.92
lnequipten:65.92
lncardmon:28.38
lncardten:28.439999999999998
lnwiremon:73.11999999999999
lnwireten:73.11999999999999


In [12]:
drop_list=[]

In [13]:
dplist=['addresscat','agecat','card2tenure','commutecat','edcat','equipmon','inccat','lnlongten',
            'longten', 'spoused','spousedcat','tenure']

print("Variables dropped--")
for var in dplist:
    if var in cat_var:
        drop_list.append(var)
    else:       
        print(var)
        list_update(var)
        credit.drop(var,axis=1,inplace=True)

Variables dropped--
equipmon
lnlongten
longten
spoused
tenure


In [14]:
for col in credit:
    dt = credit[col].dtype 
    if dt == int or dt == float:
        credit[col].fillna(credit[col].mean(),inplace=True)

In [15]:
credit.isnull().sum().value_counts()

0    116
dtype: int64

In [16]:
## Factor Analysis :--

In [17]:
credit_num=credit[numerical_var]
factor=FactorAnalysis(14).fit(credit_num)
F_analysis=pd.DataFrame(factor.components_,columns=numerical_var).T
F_analysis=F_analysis.add_prefix('Factor')

In [18]:
F_analysis

,Factor0,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13
age,4.877401,5.786633,2.806879,-0.583570,2.025639,-13.115450,-7.998749,2.984390,-0.474836,0.047928,0.130183,-0.084553,0.013382,-0.043155
ed,0.649466,-1.096663,0.247879,-0.021050,0.503601,0.268807,0.203379,0.400798,0.160844,-0.218332,0.161361,0.424352,-0.162854,0.111496
income,16.311064,3.329312,-3.324372,0.134964,52.665078,0.599744,0.138449,-0.325307,-0.065786,1.623736,-0.036180,0.048061,-0.009907,-0.048971
lninc,0.207470,0.023343,-0.047624,-0.014390,0.598095,0.025344,0.016580,0.053113,-0.005075,-0.383086,0.008140,-0.012682,0.003530,0.015365
debtinc,0.073988,0.004465,0.065402,-0.009256,0.121463,-3.526620,5.322199,-0.017015,0.022734,0.040883,0.002206,0.012377,0.006771,-0.005725
creddebt,0.593226,0.150992,-0.179685,0.027849,2.180717,-0.623761,1.038805,-0.103623,-0.007989,0.489315,-0.031605,-0.046465,-0.012788,-0.099036
lncreddebt,0.197197,0.006977,-0.030790,-0.019651,0.614651,-0.368799,0.578259,0.035371,-0.002067,-0.362149,-0.011279,-0.002985,0.008211,0.033165
othdebt,1.191027,0.205150,-0.288488,0.119749,3.753582,-1.221303,2.026593,-0.114371,0.046104,0.426156,-0.001787,0.009317,0.011318,-0.097324
lnothdebt,0.213198,0.023856,-0.040301,-0.007706,0.605576,-0.364889,0.587440,0.055471,-0.005820,-0.384058,0.013139,-0.002106,0.007676,0.016731
reside,-0.047564,-0.074489,-0.059330,-0.010720,-0.089156,0.390221,0.222098,-0.129003,0.088460,0.011400,-0.026127,-0.036755,0.040770,-0.023125


In [19]:
print('Numerical variables  :{}'.format(len(numerical_var)))
print('Categorical variables:{}'.format(len(cat_var)))

Numerical variables  :33
Categorical variables:83


In [20]:
## Dummy Creation for Categorical variables :--

In [21]:
cat_to_dum=['region','townsize','gender','agecat','edcat','empcat','inccat','jobsat','spousedcat',
         'hometype','addresscat','cars','carown','cartype','carcatvalue',
         'carbought','commutecat','reason','polview','card','cardtype','cardbenefit',
         'cardtenurecat','card2','card2type','card2benefit','card2tenurecat',
         'bfast','internet']

credit['townsize']=credit['townsize'].astype(int)
dum_var=[]  
for var in cat_to_dum:
    if var in credit:
        col_dummies = pd.get_dummies(credit[var], prefix=var+'_dummy')
        credit=pd.concat([credit,col_dummies],axis=1)
        if var not in drop_list:
            drop_list.append(var)
        for col in list(col_dummies.columns):
            dum_var.append(col)

In [22]:
print (('Numerical variables  :{}'.format(len(numerical_var))))
print (('Categorical variables:{}'.format(len(cat_var))))

Numerical variables  :33
Categorical variables:83


In [23]:
list_drop(drop_list)

In [24]:
dplist=['income','creddebt','othdebt']
# These variables are dropped as they are redundant variables    
list_drop(dplist)

In [25]:
print (('Numerical variables  :{}'.format(len(numerical_var))))
print (('Categorical variables:{}'.format(len(cat_var))))

Numerical variables  :30
Categorical variables:53


In [26]:
corr_list=dum_var + cat_var
credit_corr=credit[corr_list]
corr=pd.DataFrame(credit.corr())
corr_spent=pd.DataFrame(corr['total_spent_ln'])
corr_spent.head()

,total_spent_ln
age,0.002318
ed,0.097506
jobcat,-0.009458
union,0.022655
employ,0.066625


In [27]:
credit.columns.tolist()

['age',
 'ed',
 'jobcat',
 'union',
 'employ',
 'retire',
 'lninc',
 'debtinc',
 'lncreddebt',
 'lnothdebt',
 'default',
 'marital',
 'reside',
 'pets',
 'pets_cats',
 'pets_dogs',
 'pets_birds',
 'pets_reptiles',
 'pets_small',
 'pets_saltfish',
 'pets_freshfish',
 'homeown',
 'address',
 'carvalue',
 'carbuy',
 'commute',
 'commutetime',
 'commutecar',
 'commutemotorcycle',
 'commutecarpool',
 'commutebus',
 'commuterail',
 'commutepublic',
 'commutebike',
 'commutewalk',
 'commutenonmotor',
 'telecommute',
 'polparty',
 'polcontrib',
 'vote',
 'cardfee',
 'cardtenure',
 'card2fee',
 'carditems',
 'card2items',
 'active',
 'churn',
 'longmon',
 'lnlongmon',
 'tollfree',
 'tollmon',
 'tollten',
 'equip',
 'equipten',
 'callcard',
 'cardmon',
 'cardten',
 'wireless',
 'wiremon',
 'wireten',
 'multline',
 'voice',
 'pager',
 'callid',
 'callwait',
 'forward',
 'confer',
 'ebill',
 'owntv',
 'hourstv',
 'ownvcr',
 'owndvd',
 'owncd',
 'ownpda',
 'ownpc',
 'ownipod',
 'owngame',
 'ownfax'

In [28]:
credit.columns=['age',
 'ed',
 'jobcat',
 'union',
 'employ',
 'retire',
 'lninc',
 'debtinc',
 'lncreddebt',
 'lnothdebt',
 'default',
 'marital',
 'reside',
 'pets',
 'pets_cats',
 'pets_dogs',
 'pets_birds',
 'pets_reptiles',
 'pets_small',
 'pets_saltfish',
 'pets_freshfish',
 'homeown',
 'address',
 'carvalue',
 'carbuy',
 'commute',
 'commutetime',
 'commutecar',
 'commutemotorcycle',
 'commutecarpool',
 'commutebus',
 'commuterail',
 'commutepublic',
 'commutebike',
 'commutewalk',
 'commutenonmotor',
 'telecommute',
 'polparty',
 'polcontrib',
 'vote',
 'cardfee',
 'cardtenure',
 'card2fee',
 'carditems',
 'card2items',
 'active',
 'churn',
 'longmon',
 'lnlongmon',
 'tollfree',
 'tollmon',
 'tollten',
 'equip',
 'equipten',
 'callcard',
 'cardmon',
 'cardten',
 'wireless',
 'wiremon',
 'wireten',
 'multline',
 'voice',
 'pager',
 'callid',
 'callwait',
 'forward',
 'confer',
 'ebill',
 'owntv',
 'hourstv',
 'ownvcr',
 'owndvd',
 'owncd',
 'ownpda',
 'ownpc',
 'ownipod',
 'owngame',
 'ownfax',
 'news',
 'response_01',
 'response_02',
 'response_03',
 'total_spent_ln',
 'region_dummy_1',
 'region_dummy_2',
 'region_dummy_3',
 'region_dummy_4',
 'region_dummy_5',
 'townsize_dummy_1',
 'townsize_dummy_2',
 'townsize_dummy_3',
 'townsize_dummy_4',
 'townsize_dummy_5',
 'gender_dummy_0',
 'gender_dummy_1',
 'agecat_dummy_2',
 'agecat_dummy_3',
 'agecat_dummy_4',
 'agecat_dummy_5',
 'agecat_dummy_6',
 'edcat_dummy_1',
 'edcat_dummy_2',
 'edcat_dummy_3',
 'edcat_dummy_4',
 'edcat_dummy_5',
 'empcat_dummy_1',
 'empcat_dummy_2',
 'empcat_dummy_3',
 'empcat_dummy_4',
 'empcat_dummy_5',
 'inccat_dummy_1',
 'inccat_dummy_2',
 'inccat_dummy_3',
 'inccat_dummy_4',
 'inccat_dummy_5',
 'jobsat_dummy_1',
 'jobsat_dummy_2',
 'jobsat_dummy_3',
 'jobsat_dummy_4',
 'jobsat_dummy_5',
 'spousedcat_dummy__1',
 'spousedcat_dummy_1',
 'spousedcat_dummy_2',
 'spousedcat_dummy_3',
 'spousedcat_dummy_4',
 'spousedcat_dummy_5',
 'hometype_dummy_1',
 'hometype_dummy_2',
 'hometype_dummy_3',
 'hometype_dummy_4',
 'addresscat_dummy_1',
 'addresscat_dummy_2',
 'addresscat_dummy_3',
 'addresscat_dummy_4',
 'addresscat_dummy_5',
 'cars_dummy_0',
 'cars_dummy_1',
 'cars_dummy_2',
 'cars_dummy_3',
 'cars_dummy_4',
 'cars_dummy_5',
 'cars_dummy_6',
 'cars_dummy_7',
 'cars_dummy_8',
 'carown_dummy__1',
 'carown_dummy_0',
 'carown_dummy_1',
 'cartype_dummy__1',
 'cartype_dummy_0',
 'cartype_dummy_1',
 'carcatvalue_dummy__1',
 'carcatvalue_dummy_1',
 'carcatvalue_dummy_2',
 'carcatvalue_dummy_3',
 'carbought_dummy__1',
 'carbought_dummy_0',
 'carbought_dummy_1',
 'commutecat_dummy_1',
 'commutecat_dummy_2',
 'commutecat_dummy_3',
 'commutecat_dummy_4',
 'commutecat_dummy_5',
 'reason_dummy_1',
 'reason_dummy_2',
 'reason_dummy_3',
 'reason_dummy_4',
 'reason_dummy_9',
 'polview_dummy_1',
 'polview_dummy_2',
 'polview_dummy_3',
 'polview_dummy_4',
 'polview_dummy_5',
 'polview_dummy_6',
 'polview_dummy_7',
 'card_dummy_1',
 'card_dummy_2',
 'card_dummy_3',
 'card_dummy_4',
 'card_dummy_5',
 'cardtype_dummy_1',
 'cardtype_dummy_2',
 'cardtype_dummy_3',
 'cardtype_dummy_4',
 'cardbenefit_dummy_1',
 'cardbenefit_dummy_2',
 'cardbenefit_dummy_3',
 'cardbenefit_dummy_4',
 'cardtenurecat_dummy_1',
 'cardtenurecat_dummy_2',
 'cardtenurecat_dummy_3',
 'cardtenurecat_dummy_4',
 'cardtenurecat_dummy_5',
 'card2_dummy_1',
 'card2_dummy_2',
 'card2_dummy_3',
 'card2_dummy_4',
 'card2_dummy_5',
 'card2type_dummy_1',
 'card2type_dummy_2',
 'card2type_dummy_3',
 'card2type_dummy_4',
 'card2benefit_dummy_1',
 'card2benefit_dummy_2',
 'card2benefit_dummy_3',
 'card2benefit_dummy_4',
 'card2tenurecat_dummy_1',
 'card2tenurecat_dummy_2',
 'card2tenurecat_dummy_3',
 'card2tenurecat_dummy_4',
 'card2tenurecat_dummy_5',
 'bfast_dummy_1',
 'bfast_dummy_2',
 'bfast_dummy_3',
 'internet_dummy_0',
 'internet_dummy_1',
 'internet_dummy_2',
 'internet_dummy_3',
 'internet_dummy_4']

In [29]:
feature_list=['age',
 'ed',
 'jobcat',
 'union',
 'employ',
 'retire',
 'lninc',
 'debtinc',
 'lncreddebt',
 'lnothdebt',
 'default',
 'marital',
 'reside',
 'pets',
 'pets_cats',
 'pets_dogs',
 'pets_birds',
 'pets_reptiles',
 'pets_small',
 'pets_saltfish',
 'pets_freshfish',
 'homeown',
 'address',
 'carvalue',
 'carbuy',
 'commute',
 'commutetime',
 'commutecar',
 'commutemotorcycle',
 'commutecarpool',
 'commutebus',
 'commuterail',
 'commutepublic',
 'commutebike',
 'commutewalk',
 'commutenonmotor',
 'telecommute',
 'polparty',
 'polcontrib',
 'vote',
 'cardfee',
 'cardtenure',
 'card2fee',
 'carditems',
 'card2items',
 'active',
 'churn',
 'longmon',
 'lnlongmon',
 'tollfree',
 'tollmon',
 'tollten',
 'equip',
 'equipten',
 'callcard',
 'cardmon',
 'cardten',
 'wireless',
 'wiremon',
 'wireten',
 'multline',
 'voice',
 'pager',
 'callid',
 'callwait',
 'forward',
 'confer',
 'ebill',
 'owntv',
 'hourstv',
 'ownvcr',
 'owndvd',
 'owncd',
 'ownpda',
 'ownpc',
 'ownipod',
 'owngame',
 'ownfax',
 'news',
 'response_01',
 'response_02',
 'response_03',
 'total_spent_ln',
 'region_dummy_1',
 'region_dummy_2',
 'region_dummy_3',
 'region_dummy_4',
 'region_dummy_5',
 'townsize_dummy_1',
 'townsize_dummy_2',
 'townsize_dummy_3',
 'townsize_dummy_4',
 'townsize_dummy_5',
 'gender_dummy_0',
 'gender_dummy_1',
 'agecat_dummy_2',
 'agecat_dummy_3',
 'agecat_dummy_4',
 'agecat_dummy_5',
 'agecat_dummy_6',
 'edcat_dummy_1',
 'edcat_dummy_2',
 'edcat_dummy_3',
 'edcat_dummy_4',
 'edcat_dummy_5',
 'empcat_dummy_1',
 'empcat_dummy_2',
 'empcat_dummy_3',
 'empcat_dummy_4',
 'empcat_dummy_5',
 'inccat_dummy_1',
 'inccat_dummy_2',
 'inccat_dummy_3',
 'inccat_dummy_4',
 'inccat_dummy_5',
 'jobsat_dummy_1',
 'jobsat_dummy_2',
 'jobsat_dummy_3',
 'jobsat_dummy_4',
 'jobsat_dummy_5',
 'spousedcat_dummy__1',
 'spousedcat_dummy_1',
 'spousedcat_dummy_2',
 'spousedcat_dummy_3',
 'spousedcat_dummy_4',
 'spousedcat_dummy_5',
 'hometype_dummy_1',
 'hometype_dummy_2',
 'hometype_dummy_3',
 'hometype_dummy_4',
 'addresscat_dummy_1',
 'addresscat_dummy_2',
 'addresscat_dummy_3',
 'addresscat_dummy_4',
 'addresscat_dummy_5',
 'cars_dummy_0',
 'cars_dummy_1',
 'cars_dummy_2',
 'cars_dummy_3',
 'cars_dummy_4',
 'cars_dummy_5',
 'cars_dummy_6',
 'cars_dummy_7',
 'cars_dummy_8',
 'carown_dummy__1',
 'carown_dummy_0',
 'carown_dummy_1',
 'cartype_dummy__1',
 'cartype_dummy_0',
 'cartype_dummy_1',
 'carcatvalue_dummy__1',
 'carcatvalue_dummy_1',
 'carcatvalue_dummy_2',
 'carcatvalue_dummy_3',
 'carbought_dummy__1',
 'carbought_dummy_0',
 'carbought_dummy_1',
 'commutecat_dummy_1',
 'commutecat_dummy_2',
 'commutecat_dummy_3',
 'commutecat_dummy_4',
 'commutecat_dummy_5',
 'reason_dummy_1',
 'reason_dummy_2',
 'reason_dummy_3',
 'reason_dummy_4',
 'reason_dummy_9',
 'polview_dummy_1',
 'polview_dummy_2',
 'polview_dummy_3',
 'polview_dummy_4',
 'polview_dummy_5',
 'polview_dummy_6',
 'polview_dummy_7',
 'card_dummy_1',
 'card_dummy_2',
 'card_dummy_3',
 'card_dummy_4',
 'card_dummy_5',
 'cardtype_dummy_1',
 'cardtype_dummy_2',
 'cardtype_dummy_3',
 'cardtype_dummy_4',
 'cardbenefit_dummy_1',
 'cardbenefit_dummy_2',
 'cardbenefit_dummy_3',
 'cardbenefit_dummy_4',
 'cardtenurecat_dummy_1',
 'cardtenurecat_dummy_2',
 'cardtenurecat_dummy_3',
 'cardtenurecat_dummy_4',
 'cardtenurecat_dummy_5',
 'card2_dummy_1',
 'card2_dummy_2',
 'card2_dummy_3',
 'card2_dummy_4',
 'card2_dummy_5',
 'card2type_dummy_1',
 'card2type_dummy_2',
 'card2type_dummy_3',
 'card2type_dummy_4',
 'card2benefit_dummy_1',
 'card2benefit_dummy_2',
 'card2benefit_dummy_3',
 'card2benefit_dummy_4',
 'card2tenurecat_dummy_1',
 'card2tenurecat_dummy_2',
 'card2tenurecat_dummy_3',
 'card2tenurecat_dummy_4',
 'card2tenurecat_dummy_5',
 'bfast_dummy_1',
 'bfast_dummy_2',
 'bfast_dummy_3',
 'internet_dummy_0',
 'internet_dummy_1',
 'internet_dummy_2',
 'internet_dummy_3',
 'internet_dummy_4']

In [30]:
feature_list.index("total_spent_ln")

82

In [31]:
del feature_list[82]

In [32]:
features_new = "+".join(feature_list)

In [33]:
## Outlier Treatment :--

In [34]:
credit = credit.apply(lambda x: x.clip(lower = x.quantile(0.1), upper = x.quantile(0.9)))

In [35]:
## Treating Multi-Collinearity :--

In [36]:
y, X = dmatrices('total_spent_ln~'+features_new,credit, return_type='dataframe')

In [37]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

C:\Users\kaust\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\kaust\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\kaust\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [38]:
vif.sort_values(by=['VIF Factor'],ascending=False)

,VIF Factor,features
99,inf,agecat_dummy_6
145,inf,carown_dummy_0
133,inf,addresscat_dummy_4
132,inf,addresscat_dummy_3
131,inf,addresscat_dummy_2
...,...,...
173,NaN,polview_dummy_7
178,NaN,card_dummy_5
187,NaN,cardtenurecat_dummy_1
196,NaN,card2_dummy_5


In [39]:
x=vif[vif['VIF Factor']<10]

In [40]:
## Taking variables with VIF less than 10 as they are relevant.

In [41]:
x["features"].to_list()

['Intercept',
 'jobcat',
 'union',
 'retire',
 'debtinc',
 'lncreddebt',
 'lnothdebt',
 'default',
 'reside',
 'pets',
 'pets_cats',
 'pets_dogs',
 'pets_freshfish',
 'homeown',
 'carbuy',
 'commutetime',
 'commutecar',
 'commutemotorcycle',
 'commutecarpool',
 'commutebus',
 'commuterail',
 'commutebike',
 'commutewalk',
 'telecommute',
 'polparty',
 'polcontrib',
 'vote',
 'cardfee',
 'card2fee',
 'carditems',
 'card2items',
 'active',
 'churn',
 'tollfree',
 'equip',
 'equipten',
 'callcard',
 'multline',
 'voice',
 'pager',
 'callid',
 'callwait',
 'forward',
 'confer',
 'ebill',
 'owntv',
 'hourstv',
 'ownvcr',
 'owndvd',
 'owncd',
 'ownpda',
 'ownpc',
 'ownipod',
 'owngame',
 'ownfax',
 'news',
 'response_02',
 'response_03',
 'edcat_dummy_3',
 'edcat_dummy_4',
 'inccat_dummy_3',
 'inccat_dummy_4',
 'spousedcat_dummy_1',
 'spousedcat_dummy_2',
 'spousedcat_dummy_3',
 'hometype_dummy_1',
 'hometype_dummy_2',
 'hometype_dummy_3',
 'cars_dummy_1',
 'cars_dummy_2',
 'cars_dummy_3',
 

In [42]:
final_new=credit[[ 'ed',
 'jobcat',
 'debtinc',
 'lncreddebt',
 'lnothdebt',
 'default',
 'reside',
 'pets',
 'pets_cats',
 'pets_dogs',
 'pets_freshfish',
 'homeown',
 'carbuy',
 'commutetime',
 'commutecar',
 'commutecarpool',
 'commutebus',
 'commuterail',
 'commutewalk',
 'polparty',
 'polcontrib',
 'vote',
 'carditems',
 'card2items',
 'active',
 'churn',
 'equip',
 'equipten',
 'callcard',
 'multline',
 'voice',
 'pager',
 'callid',
 'callwait',
 'forward',
 'confer',
 'ebill',
 'owntv',
 'hourstv',
 'ownvcr',
 'owndvd',
 'owncd',
 'ownpda',
 'ownpc',
 'ownipod',
 'owngame',
 'news',
 'region_dummy_1',
 'region_dummy_3',
 'region_dummy_5',
 'townsize_dummy_1',
 'townsize_dummy_2',
 'agecat_dummy_4',
 'edcat_dummy_2',
 'edcat_dummy_3',
 'edcat_dummy_4',
 'empcat_dummy_1',
 'empcat_dummy_2',
 'empcat_dummy_5',
 'inccat_dummy_2',
 'jobsat_dummy_2',
 'jobsat_dummy_3',
 'jobsat_dummy_4',
 'hometype_dummy_1',
 'hometype_dummy_2',
 'addresscat_dummy_3',
 'cars_dummy_1',
 'cars_dummy_2',
 'cars_dummy_3',
 'carown_dummy_1',
 'carcatvalue_dummy_2',
 'reason_dummy_9',
 'polview_dummy_4',
 'card_dummy_2',
 'card_dummy_3',
 'card_dummy_4',
 'cardtenurecat_dummy_5',
 'card2_dummy_2',
 'card2_dummy_3',
 'card2_dummy_4',
 'card2tenurecat_dummy_2',
 'card2tenurecat_dummy_5',
 'internet_dummy_0',"total_spent_ln"]]

In [43]:
## Building Models :--

In [44]:
feature_columns = final_new.columns.difference( ['total_spent_ln'] )
train_X, test_X, train_y, test_y = train_test_split(final_new[feature_columns],
                                                  final_new['total_spent_ln'],
                                                  test_size=0.3,
                                                  random_state=56789)

In [45]:
## Linear Regression :--

In [46]:
clf_linear= LinearRegression()
clf_linear.fit(train_X, train_y)
y_pred_linear=clf_linear.predict(test_X)
acc_linear=round( clf_linear.score(test_X,test_y) * 100, 2)
print ('Accuracy score:'+str(acc_linear) + ' percent')

Accuracy score:60.7 percent


In [47]:
acc_linear=round( clf_linear.score(train_X, train_y) * 100, 2)
print ('Accuracy score:'+str(acc_linear) + ' percent')

Accuracy score:61.09 percent


In [48]:
print("Mean Absolute error:",round(metrics.mean_absolute_error(test_y,y_pred_linear),2))
print ('Mean Squared error:',round(mse(y_pred_linear,test_y),3))
print ('R Square:',round(r2(test_y,y_pred_linear),3))

Mean Absolute error: 0.25
Mean Squared error: 0.097
R Square: 0.607


In [49]:
## Random Forest Regressor:--

In [50]:
clf_rf= RandomForestRegressor(n_estimators=500)
clf_rf.fit(train_X, train_y)
y_pred_rf= clf_rf.predict(test_X)
acc_rf = round( clf_rf.score(test_X,test_y) * 100, 2)
print ('Accuracy score:'+str(acc_rf) + ' percent')

Accuracy score:53.83 percent


In [51]:
acc_rf=round( clf_rf.score(train_X, train_y) * 100, 2)
print ("Accuracy score:"+str(acc_rf) + ' percent')

Accuracy score:93.35 percent


In [52]:
print("Mean Absolute error:",round(metrics.mean_absolute_error(test_y,y_pred_rf),2))
print ('Mean Squared error:',round(mse(y_pred_rf,test_y),2))
print ('R Square:',round(r2(test_y,y_pred_rf),2))

Mean Absolute error: 0.27
Mean Squared error: 0.11
R Square: 0.54


In [53]:
## Ridge Regression:--

In [54]:
regressor = RidgeCV(alphas=[1, 1e3, 1e6], store_cv_values=True)

In [55]:
regressor.fit(train_X, train_y)

RidgeCV(alphas=[1, 1000.0, 1000000.0], cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=True)

In [56]:
rigdecv_pred=regressor.predict(test_X)

In [57]:
print("Accuracy score:",regressor.score(test_X, test_y).round(3))

Accuracy score: 0.607


In [58]:
print("Accuracy score:",regressor.score(train_X, train_y).round(2))

Accuracy score: 0.61


In [59]:
print("Mean absolute error:",round(metrics.mean_absolute_error(test_y,rigdecv_pred),2))
print ('Mean Squared error:',round(mse(rigdecv_pred,test_y),2))
print ('R Square:',round(r2(test_y,rigdecv_pred),3))

Mean absolute error: 0.25
Mean Squared error: 0.1
R Square: 0.607


In [71]:
## Model Output:--

In [72]:
lm=sm.OLS(train_y,train_X).fit()

In [74]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         total_spent_ln   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     67.97
Date:                Sat, 04 Jul 2020   Prob (F-statistic):               0.00
Time:                        15:24:49   Log-Likelihood:                -971.11
No. Observations:                3500   AIC:                             2102.
Df Residuals:                    3420   BIC:                             2595.
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
active                     0.0002      0.011      0.020      0.984      -0.022       0.022
addresscat_dummy_3         0.0242      0.014      1.733      0.083      -0.003       0.052
agecat_dummy_4            -0.0047      0.014     -0.322      0.748      -0.033       0.024
callcard                  -0.0133      0.015     -0.874      0.382      -0.043       0.017
callid                     0.0180      0.016      1.118      0.264      -0.014       0.050
callwait                   0.0001      0.016      0.007      0.994      -0.031       0.031
carbuy                     0.0120      0.012      1.039      0.299      -0.011       0.035
carcatvalue_dummy_2        0.0026      0.014      0.189      0.850      -0.024       0.029
card2_dummy_2             -0.1921      0.017    -11.619      0.000      -0.224      -0.160
card2_dummy_3             -0.1891      0.016    -11.547      0.000      -0.221      -0.157
card2_dummy_4             -0.1543      0.018     -8.707      0.000      -0.189      -0.120
card2items                 0.0963      0.003     37.606      0.000       0.091       0.101
card2tenurecat_dummy_2    -0.0122      0.016     -0.759      0.448      -0.044       0.019
card2tenurecat_dummy_5    -0.0089      0.021     -0.432      0.666      -0.049       0.031
card_dummy_2              -0.3429      0.017    -20.450      0.000      -0.376      -0.310
card_dummy_3              -0.3367      0.017    -19.779      0.000      -0.370      -0.303
card_dummy_4              -0.3390      0.017    -19.681      0.000      -0.373      -0.305
carditems                  0.0692      0.002     35.304      0.000       0.065       0.073
cardtenurecat_dummy_5     -0.0109      0.021     -0.525      0.600      -0.052       0.030
carown_dummy_1             0.0248      0.015      1.671      0.095      -0.004       0.054
cars_dummy_1              -0.0335      0.018     -1.857      0.063      -0.069       0.002
cars_dummy_2              -0.0183      0.016     -1.149      0.251      -0.049       0.013
cars_dummy_3              -0.0294      0.018     -1.663      0.096      -0.064       0.005
churn                      0.0308      0.014      2.128      0.033       0.002       0.059
commutebus                 0.0153      0.011      1.330      0.184      -0.007       0.038
commutecar                -0.0005      0.015     -0.031      0.975      -0.030       0.029
commutecarpool             0.0027      0.013      0.215      0.829      -0.022       0.027
commuterail               -0.0105      0.012     -0.841      0.400      -0.035       0.014
commutetime                0.0010      0.001      0.804      0.421      -0.001       0.004
commutewalk               -0.0100      0.012     -0.856      0.392      -0.033       0.013
confer                     0.0055      0.016      0.343      0.732      -0.026  

In [ ]:
## Featue Importance :--

In [ ]:
# We will be using random forest regressor as it has a higher accuracy and display less error than the other models

In [77]:
ce=pd.DataFrame(clf_rf.feature_importances_, train_X.columns)
ce.columns=['Coefficients']
ce.sort_values('Coefficients',ascending=False,inplace=True)
ce['Coefficients']=((ce['Coefficients'])*100).round(2)
ce=ce.head(10)

In [78]:
ce

,Coefficients
card2items,26.48
carditems,19.03
lnothdebt,7.95
lncreddebt,5.23
debtinc,4.90
commutetime,2.29
hourstv,2.20
ed,1.65
pets,1.35
equipten,1.29


In [ ]:
## Factors affecting the credit spend are:--
# card2items
# carditems
# lnothdebt
# lncreddebt
# debtinc
# commutetime
# hourstv
# ed
# pets
# equipten